<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/Tutorial_03_pyAMPACT_symbolic_Annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Working with Annotations</h1>



In [1]:
# Due to package dependencies issues, you will have to restart the runtime
# after running this cell and then and re-run it a second time

from IPython.utils import io
!pip install pyampact
!pip install crim_intervals
import pyampact
import pandas as pd
import crim_intervals as crim
import numpy as np
import librosa

  Using cached music21-9.1.0-py3-none-any.whl.metadata (4.8 kB)
Using cached music21-9.1.0-py3-none-any.whl (22.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Attempting uninstall: music21
    Found existing installation: music21 8.3.0
    Uninstalling music21-8.3.0:
      Successfully uninstalled music21-8.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crim-intervals 2.0.50 requires joblib==1.0.1, but you have joblib 1.5.1 which is incompatible.
crim-intervals 2.0.50 requires music21==8.3.0, but you have music21 9.1.0 which is incompatible.
  Using cached joblib-1.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached music21-8.3.0-py3-none-any.whl.metadata (4.8 kB)


In [2]:
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git

Cloning into 'pyAMPACTtutorials'...
remote: Enumerating objects: 1016, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 1016 (delta 106), reused 97 (delta 78), pack-reused 869 (from 1)
Receiving objects: 100% (1016/1016), 147.53 MiB | 16.60 MiB/s, done.
Resolving deltas: 100% (458/458), done.
Updating files: 100% (554/554), done.


<h2>Working with Humdrum Annotations<h2>

**The Humdrum analysis must be in the same .krn file as the score itself**. So in addition to having as many **kern spines as there are voices in the piece, the krn file should have at least one spine of analysis data. As a reference, there are several special spine types that we have dedicated methods for reading in as pandas dataframes in pyAMPACT. This table shows which pyAMPACT Score object methods to use to get each given .krn file spine type:

| Spine Type | pyAMPACT Methods |
|----------|----------|
| **kern | .notes(), .midiPitches() |
| **text | .lyrics() |
| **dyanm | .dynamics() |
| **harm | .harm(), .harmKeys(), .romanNumeral()|
| **function | .functions() |
| **chord | .chords() |
| **cdata | .cdata() |
| Any other spine type | .getSpines('name_of_spine_type') |

Here is an example of reading a **harm spine.

In [3]:
piece = pyampact.Score('/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.krn')
piece.harm()

array(['I', 'I', 'I', 'I', 'I', 'I', 'V7', 'V7', 'V7', 'V7', 'V7', 'I',
       'I', 'IV', 'IV', 'V', 'V', 'I', 'I', 'I'], dtype=object)

Additional parsing of .krn scores is also available. If a *function spine is in the file, you can retrieve its scoretime-aligned values with the .functions() method. For non-kern files, or for kern files without a *function spine, this method returns an empty array.

In [4]:
piece.functions()

array(['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T',
       'P', 'P', 'D', 'D', 'T', 'T', 'T'], dtype=object)

Analytic information in the symbolic files can be reshaped to match the time dimesnion of `.pianoRoll()` using the `snap_to` parameter.

In [5]:
pianoRoll = piece.pianoRoll()
piece.harm(snap_to=pianoRoll)

array(['I', 'I', 'I', 'I', 'I', 'I', 'V7', 'V7', 'V7', 'V7', 'V7', 'I',
       'I', 'IV', 'IV', 'V', 'V', 'I', 'I', 'I'], dtype=object)

The annotations can also be reshaped to the time dimension of `.mask()`. Since annotations are sparse with respect to the number of time points, functions automatically fill in the intermediary timepoints by repeating the last label. This behavior can be modified with the filler parameter.

In [6]:
mask = piece.mask(num_harmonics=4, width=1)
piece.harm(snap_to=mask)

array(['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I',
       'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I',
       'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I',
       'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I',
       'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I',
       'I', 'I', 'I', 'I', 'I', 'I', 'I', 'V7', 'V7', 'V7', 'V7', 'V7',
       'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7',
       'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7',
       'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7',
       'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7',
       'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7',
       'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7', 'V7',
       'V7', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I',
       'IV', 'IV', 'IV', 'IV', 'IV', 'IV', 'IV', 'IV', 'I

#Working with Dezran Annotations (Dezran)

pyAMPACT can import .dez annotation files and link them to corresponding score data in manner consistent to the way Humdrum annotation is imported (as described above).

In [7]:
from pyampact import snapTo
piece = pyampact.Score('/content/pyAMPACTtutorials/test_files/K279-seg.musicxml')
rn = piece.romanNumerals(output='series', dez_path='/content/pyAMPACTtutorials/test_files/K279-seg_harmony_texture.dez')

/usr/local/lib/python3.11/dist-packages/music21/musicxml/xmlToM21.py:1986: MusicXMLWarning: The following exception took place in m. 1 in part Klavier linke Hand.
  warnings.warn(


There was an issue with the function texts so they were removed.


Once the annotations have been imported, they can be snapped to a pyAMPACT score representation.

In [8]:
snapTo(rn, pianoRoll, output='series')

,tag
0.00000,I
1.00000,I
2.00000,I
2.25000,I
2.50000,I
2.75000,I
3.00000,I
3.50000,I
4.00000,ii6
5.00000,ii6


In [9]:
snapTo(rn, mask, output='series')

,tag
0.000000,I
0.041667,I
0.083333,I
0.125000,I
0.166667,I
...,...
11.791667,I
11.833333,I
11.875000,I
11.916667,I


# Working with CRIM Annotations

Renaissance dissonance analysis (from the humlib dissonant tool) can also be imported using .getSpines(), pyAMPACTs generic spine importer, to get `**cdata-rdiss` spine data.

Humdrum's Renaissance dissonance classification tool is part of humlib. It is accessible in the command-line Humdrum tools and in the Verovio Humdrum Viewer.

In [10]:
piece_url = 'https://raw.githubusercontent.com/jcdevaney/pyAMPACTtutorials/main/test_files/O_Virgo_Miserere.krn'
#piece_url = 'O_Virgo_Miserere.krn'
pyamp_piece = pyampact.Score(piece_url)
rdiss = pyamp_piece.getSpines('cdata-rdiss')
rdiss

,Voice,Part-2,Part-3
,,,
6.0,NaN,NaN,q
9.0,NaN,NaN,p
11.0,NaN,NaN,p
14.0,q,NaN,NaN
20.0,g,s,g
29.0,p,NaN,NaN
37.0,P,NaN,P
59.0,P,NaN,NaN
67.0,NaN,NaN,P


CRIM's CVF (Cadential Voice Function analysis) dataframe can be treated as if it's a pyAMPACT dataframe without any special treatment because the dataframe's index is time-aligned and the columnar index consists of part names.

In a couple of cases, CRIM dataframes differ from this structure (for example .cadences() has different columns and .presentationTypes() has a different index and columns). In these cases, the CRIM dataframes need to be restructured before importing in pyAMPACT.

The code in next cell generates an MEI file called "pyAMPACT-CRIM_test.mei.xml"  in `/` (viewable on the left under Files. It will be too big to view in Colab and should download for viewing.

In [11]:
crim_piece = crim.importScore(piece_url)

In [12]:
cvf_table = crim_piece.cvfs()
pyamp_piece.toMEI('pyAMPACT-CRIM_test', ' ', dfs={'CVF': cvf_table}, analysis_tag='harm')

pyAMPACT can also create links to short excerpts of pieces, with or without extra data. We do this in the .krn format for its character economy and extensibility, via the (Verovio Humdrum Viewer)[https://verovio.humdrum.org]. The following code shows the cadential voice functions used in the last cadence of this piece which is in the last two measures.

In [13]:
pyamp_piece.show(start=45)

https://verovio.humdrum.org/?t=ISEhQ09NOiBUaW5jdG9yaXMsIEpvaGFubmVzCiEhIU9UTDogTyB2aXJnbyBtaXNlcmVyZSBtZWkKKiprZXJuCSoqa2VybgkqKmtlcm4KKnBhcnQzCSpwYXJ0MgkqcGFydDEKKnN0YWZmMwkqc3RhZmYyCSpzdGFmZjEKKkl2b3gJKkl2b3gJKkl2b3gKKkkiUGFydC0zCSpJIlBhcnQtMgkqSSJWb2ljZQoqSSdQCSpJJ1AJKkknVgo9NDUJPTQ1CT00NQoxQUFdCTRlXQk0Y2NdCi4JNGQJNGIKLgkyYwkxYQoxRQkxQgkuCi4JLgkyZyMKPTQ2CT00Ngk9NDYKMEFBCTBBCTBhCj09CT09CT09CiotCSotCSotCiEhIVJERioqa2VybjogJT1yYXRpb25hbCByaHl0aG0KISEhUkRGKiprZXJuOiBsPWxvbmcgbm90ZSBpbiBvcmlnaW5hbCBub3RhdGlvbgohISFSREYqKmtlcm46IGk9ZWRpdG9yaWFsIGFjY2lkZW50YWwKISEhT05COiBUcmFuc2xhdGVkIGZyb20gYSBrcm4gZmlsZSBvbiAyMDI1LTA1LTI5IHZpYSBweUFNUEFDVAohISF0aXRsZTogQHtPVEx9
